In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110580

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,149625.1431
2018-02-28,161857.0168
2018-03-31,142523.7747
2018-04-30,166962.1527
2018-05-31,174388.0562
2018-06-30,194042.1202
2018-07-31,219657.3779
2018-08-31,255084.2298
2018-09-30,223945.0987


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    149625.1431
 2018-02-28    161857.0168
 2018-03-31    142523.7747
 2018-04-30    166962.1527
 2018-05-31    174388.0562
 2018-06-30    194042.1202
 2018-07-31    219657.3779
 2018-08-31    255084.2298
 2018-09-30    223945.0987
 2018-10-31    197316.9328
 2018-11-30    172126.2292
 2018-12-31    185979.5682
 2019-01-31    227946.2994
 2019-02-28    238171.5834
 2019-03-31    225982.9950
 2019-04-30    252722.7775
 2019-05-31    309635.3634
 2019-06-30    353738.6862
 2019-07-31    324762.1969
 2019-08-31    334003.3371
 2019-09-30    265488.3774
 2019-10-31    271831.1754
 2019-11-30    261006.7001
 2019-12-31    254847.9350
 2020-01-31    262932.7680
 2020-02-29    129832.8939
 2020-03-31    109429.4726
 2020-04-30    120123.7726
 2020-05-31    159612.9494
 2020-06-30    203821.2417
 2020-07-31    285229.4870
 2020-08-31    259260.2980
 2020-09-30    161896.6908
 2020-10-31    185327.6605
 2020-11-30    128251.5868
 2020-12-31    106195.5762

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.918537
p-value : 0.323391
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.356625
p-value : 0.000004
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1057.972, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1036.706, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1030.070, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1023.706, Time=0.09 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1024.510, Time=0.09 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1022.817, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1011.350, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1011.629, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1013.217, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1015.500, Time=0.06 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1017.748, Time=0.08 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1010.974, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1013.206, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1014.725, Time=0.12 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1017.307, Time=0.09 sec
 ARIMA(3,0

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -501.492
Method:                       css-mle   S.D. of innovations          36546.586
Date:                Sat, 11 Sep 2021   AIC                           1010.983
Time:                        04:07:03   BIC                           1017.934
Sample:                             0   HQIC                          1013.531
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.005e+05   2.48e+04      8.084      0.000    1.52e+05    2.49e+05
ar.L1.y        1.0204      0.148      6.911      0.000       0.731       1.310
ar.L2.y       -0.2357      0.148     -1.588      0.1

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([174968.76803046, 183500.67012216]), array([36546.58595877, 52215.17531613]), array([[103338.77579337, 246598.76026755],
       [ 81160.80705611, 285840.53318821]]))
